## Role Models
Presented by Aldrin G. Hsu, Wesley Brian T. Sy, and Gian Martin C. Carlos



In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [55]:
dfIncome = pd.read_excel("Table 6. 2018, 2021 and 2023p Share to Total Annual Family Income, by Region, Province and HUC.xlsx", engine="openpyxl")
dfExpenditure = pd.read_excel("Table 7. 2018, 2021 and 2023p Share to Total Family Expenditure, by Region, Province and HUC.xlsx", engine="openpyxl")

In [56]:
dfExpenditure.head(10)


,"Table 7. Share to Total Family Expenditure, by Region, Province and HUC: 2018, 2021 and 2023p",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Region/Province/HUC by Expenditure Group*,2018,2021.00,2023p,2021-2018,2023-2021
1,NaN,(In percent),NaN,NaN,Percentage Point Change,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,Philippines,NaN,NaN,NaN,NaN,NaN
4,Share to Family Expenditure,100,100.00,100,NaN,NaN
5,Food and Non-Alcoholic Beverages,42.55,42.62,40.93,0.07,-1.7
6,Food Consumed at Home,33.6,38.53,35.34,4.93,-3.19
7,Food Regularly Consumed Outside the Home,8.95,4.09,5.57,-4.86,1.49
8,Service for Processing Primary Goods for F...,NaN,0.01,0.01,0.01,0
9,Alcoholic Beverages and Tobacco,1.82,1.74,1.42,-0.09,-0.32


In [96]:
#Table 6. 2018, 2021 and 2023p Share to Total Annual Family Income, by Region, Province and HUC
Income18 = pd.read_csv("I18.csv")
Income21 = pd.read_csv("I21.csv")
Income23 = pd.read_csv("I23.csv")

#Table 7. 2018, 2021 and 2023p Share to Total Family Expenditure, by Region, Province and HUC
Expenditure18 = pd.read_csv("E18.csv")
Expenditure21 = pd.read_csv("E21.csv")
Expenditure23 = pd.read_csv("E23.csv")

In [97]:
Income18 = Income18.transpose()
Income21 = Income21.transpose()
Income23 = Income23.transpose()
Expenditure18 = Expenditure18.transpose()
Expenditure21 = Expenditure21.transpose()
Expenditure23 = Expenditure23.transpose()

In [98]:
Income18.drop(labels="Maguindanao del Sur", axis=0, inplace=True)
Income18.drop(labels="Maguindanao del Norte", axis=0, inplace=True)
Income21.drop(labels="Maguindanao del Sur", axis=0, inplace=True)
Income21.drop(labels="Maguindanao del Norte", axis=0, inplace=True)
Income23.drop(labels="Maguindanao del Sur", axis=0, inplace=True)
Income23.drop(labels="Maguindanao del Norte", axis=0, inplace=True)

Expenditure18.drop(labels="Maguindanao del Sur", axis=0, inplace=True)
Expenditure18.drop(labels="Maguindanao del Norte", axis=0, inplace=True)
Expenditure21.drop(labels="Maguindanao del Sur", axis=0, inplace=True)
Expenditure21.drop(labels="Maguindanao del Norte", axis=0, inplace=True)
Expenditure23.drop(labels="Maguindanao del Sur", axis=0, inplace=True)
Expenditure23.drop(labels="Maguindanao del Norte", axis=0, inplace=True)

In [112]:
Expenditure18.iloc[1:, 1:] = (
    Expenditure18.iloc[1:, 1:]
    .replace(['-', '–', '−', '', ' -', ' - ', '▪', '▪ -', '▪-'], '0')
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
)
Expenditure21.iloc[1:, 1:] = (
    Expenditure21.iloc[1:, 1:]
    .replace(['-', '–', '−', '', ' -', ' - ', '▪', '▪ -', '▪-'], '0')
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
)
Expenditure23.iloc[1:, 1:] = (
    Expenditure23.iloc[1:, 1:]
    .replace(['-', '–', '−', '', ' -', ' - ', '▪', '▪ -', '▪-'], '0')
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
)


C:\Users\Aldrin Hsu\AppData\Local\Temp\ipykernel_17708\3277989987.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(['-', '–', '−', '', ' -', ' - ', '▪', '▪ -', '▪-'], '0')
C:\Users\Aldrin Hsu\AppData\Local\Temp\ipykernel_17708\3277989987.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(['-', '–', '−', '', ' -', ' - ', '▪', '▪ -', '▪-'], '0')


In [ ]:
Expenditure18.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Unnamed: 0,Share to Family Expenditure,Food and Non-Alcoholic Beverages,Food Consumed at Home,Food Regularly Consumed Outside the Home,Service for Processing Primary Goods for F...,Alcoholic Beverages and Tobacco,Clothing and Footwear,"Housing, Water, Electricity, Gas and Other Fuels",Furnishings and Routine Household Maintenance,Health,Transport,Information and Communication,"Recreation, Sport and Culture",Education,Hotels (Accommodation Services),Insurance and Financial Services,"Personal Care, Social Protection and Miscellan...",Durable Furniture and Equipment,Special Family Occasion,Other Expenditures
PHILIPPINES,100.0,42.6,38.5,4.1,0.0,1.7,1.5,24.1,1.9,3.3,5.6,3.5,0.4,1.5,0.1,3.1,4.4,2.5,2.0,1.8
NATIONAL CAPITAL REGION,100.0,40.1,32.7,7.4,0.0,1.3,1.0,30.9,1.9,2.5,4.8,4.1,0.4,1.3,0.0,3.1,3.6,1.9,1.0,2.2
City of Manila,100.0,47.7,35.3,12.4,0.0,1.6,1.0,26.6,1.4,2.3,3.6,3.5,0.2,1.2,0.0,2.8,3.1,1.3,1.0,2.6
City of Mandaluyong,100.0,41.1,28.6,12.5,0.0,1.5,1.3,28.7,1.4,2.0,4.2,4.0,0.2,1.1,0.1,3.2,3.1,2.2,0.9,5.1
City of San Juan,100.0,39.9,29.9,10.0,0.0,1.3,1.1,26.4,2.1,2.3,4.2,3.7,0.6,1.9,0.0,3.5,4.0,2.4,1.4,5.3
Quezon City,100.0,38.9,32.4,6.5,0.0,1.3,1.0,30.3,2.4,3.1,5.9,4.2,0.6,1.1,0.0,2.8,3.5,1.9,0.5,2.4
City of Marikina,100.0,35.2,28.7,6.5,0.0,1.1,1.0,31.2,2.3,2.9,5.6,4.4,0.4,1.7,0.0,4.2,3.5,3.6,1.0,1.8
City of Pasig,100.0,35.4,31.6,3.8,0.0,1.2,0.9,34.3,1.7,2.4,4.7,4.6,0.3,1.9,0.0,3.8,3.5,2.4,0.8,2.3
City of Caloocan,100.0,44.9,37.5,7.4,0.0,1.7,1.1,27.2,1.2,2.0,4.7,4.2,0.3,1.2,0.1,2.5,4.2,2.0,1.5,1.3
